In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.5.0

# Quick SparkSession start
import sparknlp
spark = sparknlp.start()

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 64kB/s 
     |████████████████████████████████| 204kB 40.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=64339c399963da41cdd7e7698a250491e7d9630771a23111d2ec12bac2b17551
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 122kB 3.5MB/s 
Spark NLP version
Apache Spark version


'2.4.4'

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

In [0]:
import pyspark
from pyspark.sql import *
import sparknlp
spark=sparknlp.start()
sc=pyspark.SparkContext.getOrCreate()
textfile=sc.textFile("/content/drive/My Drive/dic487-587/train.csv")

In [0]:
dataframe=spark.read.csv("/content/drive/My Drive/dic487-587/train.csv",inferSchema=True,header=True,escape='"')
dataframe.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)



Test

In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import (StopWordsRemover,HashingTF, Tokenizer,CountVectorizer)
import numpy as np
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.feature import PCA
from pyspark.ml.feature import (StopWordsRemover,HashingTF, Tokenizer,CountVectorizer)
from pyspark.ml.classification import LogisticRegression
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,Stemmer,
                                LemmatizerModel, StopWordsCleaner,PerceptronApproach,Chunker,RegexMatcher)
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline
from sparknlp.training import POS
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
eng_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
countTokens = udf(lambda words: len(words), IntegerType())
regexTokenized = regexTokenizer.transform(dataframe)
regexTokenized.select("words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
stopwordsremoved=remover.transform(regexTokenized)
stopwordsremoved.select('filtered').show(truncate=False)



+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import regexp_extract, when

mapping_csv=spark.read.load('/content/drive/My Drive/dic487-587/mapping.csv',format="csv",sep=',',inferSchema='true',header='true',escape='"')
stopwordsremoved.select('genre').show(truncate=False)
col1=mapping_csv.select('0').collect()
list1=[str(i[0]) for i in col1]   
for val in list1:
    stopwordsremoved = stopwordsremoved.withColumn(val, when(stopwordsremoved['genre'].contains("'"+ val +"'"),1.0).otherwise(0.0))

stopwordsremoved.show()


+-------------------------------------------------------------------------+
|genre                                                                    |
+-------------------------------------------------------------------------+
|['World cinema', 'Drama']                                                |
|['Action/Adventure', 'Action', 'Science Fiction', 'Drama']               |
|['Musical', 'Action', 'Drama']                                           |
|['Comedy']                                                               |
|['Crime Fiction', 'World cinema', 'Drama']                               |
|['Action/Adventure', 'Action', 'Thriller', 'Drama']                      |
|['Thriller', 'Drama', 'Horror']                                          |
|['Drama']                                                                |
|['Black-and-white', 'Comedy', 'Romance Film', 'Romantic comedy', 'Drama']|
|['Animation', 'Short Film', 'Family Film']                               |
|['Comedy'] 

In [0]:
test_csv=spark.read.load('/content/drive/My Drive/dic487-587/test.csv',format="csv",sep=',',inferSchema='true',header='true',escape='"')


test_csv.printSchema()
regexTokenizer_test = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
countTokens = udf(lambda words: len(words), IntegerType())
regexTokenized_test = regexTokenizer_test.transform(test_csv)
regexTokenized_test.select("words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

remover_test = StopWordsRemover(inputCol="words", outputCol="filtered")
stopwordsremoved_test=remover_test.transform(regexTokenized_test)
stopwordsremoved_test.select('filtered').show(truncate=False)
test_csv=stopwordsremoved_test

test_csv.show()

root
 |-- movie_id: integer (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
trainingData=stopwordsremoved
trainingData.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Drama: double (nullable = false)
 |-- Comedy: double (nullable = false)
 |-- Romance Film: double (nullable = false)
 |-- Thriller: double (nullable = false)
 |-- Action: double (nullable = false)
 |-- World cinema: double (nullable = false)
 |-- Crime Fiction: double (nullable = false)
 |-- Horror: double (nullable = false)
 |-- Black-and-white: double (nullable = false)
 |-- Indie: double (nullable = false)
 |-- Action/Adventure: double (nullable = false)
 |-- Adventure: double (nullable = false)
 |-- Family Film: double (nullable = false)
 |-- Short Film: double (nullable = false)
 |-- Romantic drama: double (nullable = false)
 |-- Animation: double (nu

**P2 using TF-IDF**

In [0]:
from pyspark.ml.feature import NGram
from pyspark.ml.classification import LinearSVC,NaiveBayes
from pyspark.ml.feature import HashingTF, IDF, Word2Vec
ngram = NGram(n=2, inputCol="filtered", outputCol="ngrams")

ngramDataFrame_train = ngram.transform(trainingData)
ngramDataFrame_train.select("ngrams").show(truncate=False)
ngramDataFrame_test = ngram.transform(test_csv)
ngramDataFrame_test.select("ngrams").show(truncate=False)

Vector = CountVectorizer(inputCol="ngrams", outputCol="features",vocabSize=10000)
model = Vector.fit(ngramDataFrame_train)
idf_trainingData = model.transform(ngramDataFrame_train)
idf_testData = model.transform(ngramDataFrame_test)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
test_result=idf_testData
test_result.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               words|            filtered|              ngrams|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|[film based, base...|(10000,[17,42,60,...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|[group teenagers,...|(10000,[102,326,8...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|[story zulu, zulu...|(10000,[337,781,1...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|[stooges play, pl...|(10000,[17,38,207...|
|24165951|        Getting Even|A soldier-

We have commented LogisticRegressionWithLBFGS.train and uploading the pre trained model.

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.util import MLUtils
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionWithSGD
import pickle
#test_result = test_result.withColumn("id",monotonically_increasing_id())
count=0
for val in list1:
    print(val)
    trainingData_val=idf_trainingData.withColumnRenamed(val,'label')
    input_train = MLUtils.convertVectorColumnsFromML(trainingData_val.select('label','features')).rdd
    input_train = input_train.map(lambda row : LabeledPoint(row.label, row.features))
    # lrm = LogisticRegressionWithLBFGS.train(input_train)

    # with open("/content/drive/My Drive/dic487-587/P3-Main-models/"+str(count),"wb")as fid: pickle.dump(lrm,fid)
    loadedmodel=pickle.load(open("/content/drive/My Drive/dic487-587/P3-Main-models/"+str(count),"rb"))

    input_test = MLUtils.convertVectorColumnsFromML(idf_testData.select('movie_id','features')).rdd
    testData_pred=input_test.map(lambda a: (a.movie_id, loadedmodel.predict(a.features)))
    sqlContext = SQLContext(sc)
    testData_val = sqlContext.createDataFrame(testData_pred)
    testData_val.createOrReplaceTempView("testData_pred")
    testData_val=testData_val.withColumnRenamed('_1','movie_id')
    testData_val=testData_val.withColumnRenamed('_2', val)
    test_result = test_result.join(testData_val, test_result.movie_id == testData_val.movie_id).select(test_result["*"],testData_val[val])
    count=count+1
# test_result.show()


Drama
Comedy
Romance Film
Thriller
Action
World cinema
Crime Fiction
Horror
Black-and-white
Indie
Action/Adventure
Adventure
Family Film
Short Film
Romantic drama
Animation
Musical
Science Fiction
Mystery
Romantic comedy


In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import concat,lit
from pyspark.sql.types import StringType
test_result=test_result.withColumn("predictions",lit("").cast(StringType()))

for val in list1:
  test_result=test_result.withColumn("predictions",concat(col("predictions") ,lit(" "), col(val)))

In [0]:
test_result.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+---------------+--------------------+
|movie_id|          movie_name|                plot|               words|            filtered|              ngrams|            features|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|Family Film|Short Film|Romantic drama|Animation|Musical|Science Fiction|Mystery|Romantic comedy|         predictions|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+--------

In [0]:
exportfile=test_result.select("movie_id","predictions")
exportfile.show(truncate=False)

+--------+----------------------------------------+
|movie_id|predictions                             |
+--------+----------------------------------------+
|62693   | 1 0 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0|
|296252  | 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|1356971 | 1 1 1 1 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1|
|1428872 | 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|1582173 | 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0|
|1595142 | 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0|
|1600825 | 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0|
|1681132 | 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0|
|2268290 | 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0|
|3327607 | 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0|
|3569957 | 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0|
|4413498 | 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0|
|4635580 | 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|4950989 | 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0|
|5565692 | 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0|
|7003785 | 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0|
|8269287 | 0

In [0]:

exportfile.coalesce(1) \
       .write \
       .csv("/content/drive/My Drive//dic487-587/Submission9",header=True)